## Download Library

In [ ]:
! pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.7 MB/s eta 0:00:00


In [ ]:
import openai

In [ ]:
! pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.8 MB/s eta 0:00:00


## Scrap `Any` PDF

We need the `PyMuPDF` package in python. So, we install it first.

In [ ]:
! pip install PyMuPDF

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 23.7 MB/s eta 0:00:00


### `read_pdf_content` function

In [ ]:
import fitz  # PyMuPDF

def read_pdf_content(pdf_path):
    """
    Reads a PDF and returns its content as a list of strings.

    Args:
    pdf_path (str): The file path to the PDF.

    Returns:
    list of str: A list where each element is the text content of a PDF page.
    """
    content_list = []
    with fitz.open(pdf_path) as doc:
        for page in doc:
            content_list.append(page.get_text())

    return content_list

In [ ]:
%%time

scraped_content = read_pdf_content("/content/all_ysa_doc.pdf")

CPU times: user 94.9 ms, sys: 2.71 ms, total: 97.6 ms
Wall time: 109 ms


In [ ]:
len(scraped_content)

57

In [ ]:
len(scraped_content[0])

237

In [ ]:
scraped_content[0]

'About YSA\nYouth Spirit Artworks (YSA) is a program in Berkeley, California, that helps homeless and\nlow-income young people in the San Francisco Bay Area. We focus on BIPOC and LGBTQIA+\ncommunities who are often in the low-income group.\n'

In [ ]:
scraped_content = ' '.join(scraped_content)

In [ ]:
%%time

scraped_content = [scraped_content.split('. ')[i].replace('\n', '').replace('   ', '').replace('  ', '') for i in range(len(scraped_content.split('. ')))]

CPU times: user 13.3 ms, sys: 4 µs, total: 13.3 ms
Wall time: 13.2 ms


In [ ]:
scraped_content[5]

'Sally has a background in theology and art, andshe also co-founded a homeless newspaper called Street Spirit.But YSA was not just Sally’s effort'

In [ ]:
len(scraped_content)

131

In [ ]:
scraped_content_backup = scraped_content

In [ ]:
scraped_content = scraped_content_backup[0:20]

## API Call to Create Data

Here we use the `client.chat.completions.create` function from *OpenAI* as a helper function to assist us to create question answer.

In [ ]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [ ]:
# OPENAI_API_KEY = "sk-xxx"
openai_client = openai.OpenAI(api_key=OPENAI_API_KEY)


def call_chatgpt(query: str, model: str = "gpt-3.5-turbo") -> str:
    """
    Generates a response to a query using the specified language model.
    Args:
        query (str): The user's query that needs to be processed.
        model (str, optional): The language model to be used. Defaults to "gpt-3.5-turbo".
    Returns:
        str: The generated response to the query.
    """

    # Prepare the conversation context with system and user messages.
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": f"Question: {query}."},
    ]

    # Use the OpenAI client to generate a response based on the model and the conversation context.
    response = openai_client.chat.completions.create(
        model=model,
        messages=messages,
    )

    # Extract the content of the response from the first choice.
    content: str = response.choices[0].message.content

    # Return the generated content.
    return content

In [ ]:
resp = call_chatgpt("tell me a joke")

In [ ]:
resp

"Sure, here's a joke for you:\n\nWhy did the scarecrow win an award?\nBecause he was outstanding in his field!"

### Prompt Engineer

We use prompt engineer to ensure the content `GPT` gave us is in the same content as the `openassist/guanaco` data.

```python
    ### Human:
    ### Assistant:
```

In [ ]:
def prompt_engineered_api(text: str):

    prompt = f"""
        I have the following content: {text}

        Write one question based on the content above. Just write ONE question in a sentence. No more.
    """

    resp = call_chatgpt(prompt)

    return resp

In [ ]:
%%time

quest = scraped_content[0]
resp = prompt_engineered_api(quest)
print(f"Q: {quest}")
print(f"A: {resp}")

Q: About YSAYouth Spirit Artworks (YSA) is a program in Berkeley, California, that helps homeless andlow-income young people in the San Francisco Bay Area
A: How does Youth Spirit Artworks (YSA) support homeless and low-income young people in the San Francisco Bay Area?
CPU times: user 18.3 ms, sys: 3.04 ms, total: 21.3 ms
Wall time: 632 ms


## Create `DataDict` Structure

In [ ]:
from datasets import Dataset, DatasetDict
from tqdm import tqdm

In [ ]:
%%time

raw_content_questions = []
raw_content_answers = []
for i in tqdm(range(len(scraped_content))):
    quest = scraped_content[i]
    resp = prompt_engineered_api(quest)
    this_sample_question = resp.split("###")[0]
    this_sample_answer = scraped_content[i]
    raw_content_questions.append(this_sample_question)
    raw_content_answers.append(this_sample_answer)

100%|██████████| 20/20 [00:16<00:00,  1.20it/s]

CPU times: user 382 ms, sys: 15 ms, total: 397 ms
Wall time: 16.7 s


In [ ]:
i = 5
raw_content_questions[i], raw_content_answers[i]

('What role did Sally play in the creation of the homeless newspaper Street Spirit?',
 'Sally has a background in theology and art, andshe also co-founded a homeless newspaper called Street Spirit.But YSA was not just Sally’s effort')

In [ ]:
# Example data - replace these with your actual data
train_data = {
    'questions': raw_content_questions,
    'answers': raw_content_answers
}

# Create Dataset objects for training and testing
train_dataset = Dataset.from_dict(train_data)

# Combine them into a DatasetDict
dataset_dict = DatasetDict({
    'train': train_dataset,
})

# Display the structure of the dataset
print(dataset_dict)

DatasetDict({
    train: Dataset({
        features: ['questions', 'answers'],
        num_rows: 20
    })
})


## Push to HuggingFace Hub

In [ ]:
! huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' creden

In [ ]:
from huggingface_hub import HfApi, create_repo

In [ ]:
from google.colab import userdata
auth_token = userdata.get('HF_TOKEN')

In [ ]:
# Replace 'your_token_here' with your actual Hugging Face Auth token
# Replace 'youthless-homeless-shelter-web-scrape-dataset' with your desired repository name
# auth_token = 'hf_xxx'
repo_name = 'ysa-web-scrape-dataset-qa-formatted-small-version'
username = 'eagle0504' # replace with your Hugging Face username

api = HfApi()
create_repo(repo_name, token=auth_token, private=False) # Set private=True if you want it to be a private dataset

RepoUrl('https://huggingface.co/eagle0504/ysa-web-scrape-dataset-qa-formatted-small-version', endpoint='https://huggingface.co', repo_type='model', repo_id='eagle0504/ysa-web-scrape-dataset-qa-formatted-small-version')

In [ ]:
app_id = f"{username}/{repo_name}"
print(app_id)

eagle0504/ysa-web-scrape-dataset-qa-formatted-small-version


In [ ]:
%%time

dataset_dict.push_to_hub(app_id)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CPU times: user 83.4 ms, sys: 4.53 ms, total: 87.9 ms
Wall time: 812 ms


CommitInfo(commit_url='https://huggingface.co/datasets/eagle0504/ysa-web-scrape-dataset-qa-formatted-small-version/commit/6dfddb43d723b281659dd4260fbbcb43fe5c24ee', commit_message='Upload dataset', commit_description='', oid='6dfddb43d723b281659dd4260fbbcb43fe5c24ee', pr_url=None, pr_revision=None, pr_num=None)

## Pull Data from HuggingFace

If you are already have a `DataDict` on *HuggingFace*, you can start here and use the following code to load the data in directly to make some queries.

You can use this code directly in a `streamlit` application.

In [ ]:
! pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.0/509.0 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.1/41.1 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.6/105.6 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 8.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.9/698.9 kB 19

In [ ]:
import chromadb
from datasets import load_dataset
import numpy as np
import pandas as pd
import string

In [ ]:
%%time

# dataset = load_dataset("eagle0504/youthless-homeless-shelter-web-scrape-dataset-qa-formatted")
dataset = load_dataset("eagle0504/larkin-web-scrape-dataset-qa-formatted")
client = chromadb.Client()
random_number = np.random.randint(low=1e9, high=1e10)
random_string = ''.join(np.random.choice(list(string.ascii_uppercase + string.digits), size=10))
combined_string = f"{random_number}{random_string}"
collection = client.create_collection(combined_string)

# Embed and store the first N supports for this demo
L = len(dataset["train"]['questions'])
collection.add(
    ids=[str(i) for i in range(0, L)],  # IDs are just strings
    documents=dataset["train"]['questions'], # Enter questions here
    metadatas=[{"type": "support"} for _ in range(0, L)],
)

Generating train split:   0%|          | 0/568 [00:00<?, ? examples/s]

/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:03<00:00, 21.4MiB/s]


CPU times: user 1min 28s, sys: 2.04 s, total: 1min 30s
Wall time: 1min 42s


In [ ]:
question = "What is Samantha Greene's role?"

In [ ]:
results = collection.query(
    query_texts=question,
    n_results=5
)

In [ ]:
idx = results["ids"][0]
idx = [int(i) for i in idx]
idx

[6, 292, 23, 303, 294]

In [ ]:
ref = pd.DataFrame(
    {
        "idx": idx,
        "question": [dataset["train"]['questions'][i] for i in idx],
        "answers": [dataset["train"]['answers'][i] for i in idx],
        "distances": results["distances"][0]
    }
)
ref

,idx,question,answers,distances
0,6,What is Samantha Greene's role?,Samantha Greene is the Art Program Specialist,0.000000
1,292,Who is Christy Steiner in memory of?,and Christy Steiner (in memory of Edwin H,1.187724
2,23,What time is Sarah teaching guitar in the art ...,"(Art Studio), Guitar with Sarah from 1 to 4 p.m",1.190716
3,303,What events and activities has Marilyn been in...,Marilyn has been supporting Larkin Street as a...,1.236349
4,294,What is the relationship between Zenoff and Ja...,Zenoff and Janet N,1.275667


In [ ]:
special_threshold = 0.3
filtered_ref = ref[ref["distances"] < special_threshold]
filtered_ref

,idx,question,answers,distances
0,1,What challenges do older homeless and low-inco...,We focus on BIPOC and LGBTQIA+communities who ...,0.0
